**Install module**

In [ ]:
path = '/content/'

In [ ]:
!pip install imagehash
!pip install tqdm
!pip install pickle
!pip install sys
!pip install platform
!pip install scikit-image 
!pip install keras 
!pip install scipy
!unzip /content/train.zip
!unzip /content/test.zip

In [ ]:
### module

# module for read data
from pandas import read_csv
#from imagehash import phash
from math import sqrt
import os
from os.path import isfile
import matplotlib.pyplot as plt
from PIL import Image as pil_image
from tqdm import tqdm_notebook

import pickle
import numpy as np
from numpy.random import normal as rnorm
import random


# module for neural network
import sys
import platform
import keras


from keras import backend as K
from keras.preprocessing.image import img_to_array,array_to_img
from scipy.ndimage import affine_transform

from keras.layers import Reshape, Conv2D, Flatten, Dense
from keras import regularizers
from keras.optimizers import Adam
from keras.engine.topology import Input
from keras.models import Model
from keras.applications.resnet50 import ResNet50
from keras.applications.mobilenet import MobileNet
#from keras.applications.vgg16 import VGG16
#from keras.applications.vgg19 import VGG19
from keras.layers import Concatenate, Lambda

from keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from keras.models import Sequential


from skimage.io import imread
from skimage.transform import resize
import numpy as np
from keras.utils import Sequence


In [ ]:
down_size = 4 # kernal numer // down size: how much kernels to use
down_size_data = 5 # len(training_data) // down_size_data  to use

In [ ]:
import os
print(os.listdir("../input/"))

['whale-categorization-playground', 'p2h-table-new', 'bounding-box']


In [ ]:
def unique(list1): 
    unique_list = [] 
    for x in list1: 
        if x not in unique_list: 
            unique_list.append(x) 
    return  unique_list


**Read Data**

In [ ]:
## read the datasets
def expand_path(p):
    if isfile(path+'train/' + p): 
        return path+ 'train/' + p
    return p


tagged_list = [(p,w) for _,p,w in read_csv(path+'train.csv').to_records()]
tagged = dict(tagged_list)

all_whales = list( set(tagged.values()) )
print("n of whales:",len(all_whales))
print("n of imgs:",len(tagged_list))


temp = []
for i, val in enumerate(all_whales): 
    if sum([j==val for j in tagged.values()]) > 2:
        temp += [val]
        
print("n of whales:",len(temp))

all_whales = random.sample(temp, len(temp) // down_size_data)

print("n of whales:",len(all_whales))

temp = []
for i, val in enumerate(tagged_list): 
    if val[1] in all_whales:
        temp += [val]
print("n of imgs:",len(temp))

n_train = len(temp) // 3 * 2
tagged = dict(temp[:n_train]) # training set
submit = dict(temp[n_train:]) # testing set
print("n of imgs:",len(tagged))


join   = list(tagged.keys()) + list(submit.keys())


### Reading other datasets

# Read the bounding box data from the bounding box kernel (see reference above)
path1 = path + 'bounding-box.pickle'
with open(path1, 'rb') as f:
    p2bb = pickle.load(f)

#Delete duplicate images
# numbers: 25460 to 20913
# path2 = '../input/h2p-table/h2p_table.csv'
# h2p = dict([(p,w) for _,p,w in read_csv(path2).to_records()])
# path3 = '../input/p2htable/p2htable.csv'
# p2h = dict([(p,w) for _,p,w in read_csv(path3).to_records()])

p2size = {}
for p in tqdm_notebook(join):
    size      = pil_image.open(expand_path(p)).size
    p2size[p] = size

n of whales: 4251
n of imgs: 9850
n of whales: 997
n of whales: 199
n of imgs: 969
n of imgs: 646


**Divide into train dataset and test dataset**

In [ ]:
import pandas as pd
#train:dict to list
train_new_list=[]
for key, value in tagged.items():
    temp = [key,value]
    train_new_list.append(temp)
#train:list to df
train_new_df = pd.DataFrame(train_new_list, columns = ['Image','Id']) 
train_new_df.head()

,Image,Id
0,00eab0e6.jpg,w_886257d
1,00f4c92c.jpg,w_9ea2cc3
2,019cd867.jpg,w_0ead9d7
3,020028b2.jpg,w_0de84f0
4,0237ca7a.jpg,w_8ba2066


In [ ]:
#train:dict to list
test_new_list=[]
for key, value in submit.items():
    temp = [key,value]
    test_new_list.append(temp)
#train:list to df
test_new_df = pd.DataFrame(test_new_list, columns = ['Image','Id']) 
test_new_df.head()

,Image,Id
0,aaff3333.jpg,w_fc7cc24
1,ab09fe21.jpg,w_a59905f
2,abca61c7.jpg,w_cd02407
3,abcf9835.jpg,w_e6f5e09
4,abd863e3.jpg,w_e3c119c


In [ ]:
# Suppress annoying stderr output when importing keras.
import sys
import platform
old_stderr = sys.stderr
sys.stderr = open('/dev/null' if platform.system() != 'Windows' else 'nul', 'w')
import keras
sys.stderr = old_stderr

import random
from keras import backend as K
from keras.preprocessing.image import img_to_array,array_to_img
from scipy.ndimage import affine_transform

# show imgs
import matplotlib.pyplot as plt

def show_whale(images, per_row=2):
    # input: iamge names['0c35fcb4.jpg'] or names
    if type(images[0]) == str:
        imgs = [pil_image.open(expand_path(p)) for p in images]
    else:
        imgs = images
    n         = len(imgs)
    rows      = (n + per_row - 1)//per_row
    cols      = min(per_row, n)
    fig, axes = plt.subplots(rows,cols, figsize=(24//per_row*cols,24//per_row*rows))
    for ax in axes.flatten(): ax.axis('off')
    for i,(img,ax) in enumerate(zip(imgs, axes.flatten())): ax.imshow(img.convert('RGB'))
    return 

In [ ]:
# Read the bounding box data from the bounding box kernel (see reference above)
#path1 = '../input/bound-box/bounding-box.pickle'
#with open(path1, 'rb') as f:
#    p2bb = pickle.load(f)

#Delete duplicate images
# numbers: 25460 to 20913
path2 = path + 'h2p_table.csv'
h2p = dict([(p,w) for _,p,w in read_csv(path2).to_records()])
path3 = path + 'p2htable.csv'
p2h = dict([(p,w) for _,p,w in read_csv(path3).to_records()])



**Image Preprocessing**

In [ ]:
# one img name to the img[PIL file]

img_shape    = (384,384,1) # The image shape used by the model
anisotropy   = 2.15 # The horizontal compression ratio
crop_margin  = 0.05 # The margin added around the bounding box to 
# compensate for bounding box inaccuracy

def build_transform(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    """
    Build a transformation matrix with the specified characteristics.
    """
    rotation        = np.deg2rad(rotation)
    shear           = np.deg2rad(shear)
    rotation_matrix = np.array([[np.cos(rotation), np.sin(rotation), 0], [-np.sin(rotation), np.cos(rotation), 0], [0, 0, 1]])
    shift_matrix    = np.array([[1, 0, height_shift], [0, 1, width_shift], [0, 0, 1]])
    shear_matrix    = np.array([[1, np.sin(shear), 0], [0, np.cos(shear), 0], [0, 0, 1]])
    zoom_matrix     = np.array([[1.0/height_zoom, 0, 0], [0, 1.0/width_zoom, 0], [0, 0, 1]])
    shift_matrix    = np.array([[1, 0, -height_shift], [0, 1, -width_shift], [0, 0, 1]])
    return np.dot(np.dot(rotation_matrix, shear_matrix), np.dot(zoom_matrix, shift_matrix))


def read_cropped_image(p, augment):
    """
    @param p : the name of the picture to read
    @param augment: True/False if data augmentation should be performed
    @return a numpy array with the transformed image
    """
    # If an image id was given (wrongly), convert to filename
    if p in h2p: p = h2p[p]
    size_x,size_y = p2size[p]
    
    # Determine the region of the original image we want to capture based on the bounding box.
    x0,y0,x1,y1   = p2bb[p]
    dx            = x1 - x0
    dy            = y1 - y0
    x0           -= dx*crop_margin
    x1           += dx*crop_margin + 1
    y0           -= dy*crop_margin
    y1           += dy*crop_margin + 1
    if (x0 < 0     ): x0 = 0
    if (x1 > size_x): x1 = size_x
    if (y0 < 0     ): y0 = 0
    if (y1 > size_y): y1 = size_y
    dx            = x1 - x0
    dy            = y1 - y0
    if dx > dy*anisotropy:
        dy  = 0.5*(dx/anisotropy - dy)
        y0 -= dy
        y1 += dy
    else:
        dx  = 0.5*(dy*anisotropy - dx)
        x0 -= dx
        x1 += dx

    # Generate the transformation matrix
    trans = np.array([[1, 0, -0.5*img_shape[0]], [0, 1, -0.5*img_shape[1]], [0, 0, 1]])
    trans = np.dot(np.array([[(y1 - y0)/img_shape[0], 0, 0], [0, (x1 - x0)/img_shape[1], 0], [0, 0, 1]]), trans)
    if augment:
#         During training, data augmentation is performed by adding a random transformation
#         that composes zoom, shift, rotation and shear.
#         The random transform is skipped when testing.
        trans = np.dot(build_transform(
            random.uniform(-5, 5),
            random.uniform(-5, 5),
            random.uniform(0.8, 1.0),
            random.uniform(0.8, 1.0),
            random.uniform(-0.05*(y1 - y0), 0.05*(y1 - y0)),
            random.uniform(-0.05*(x1 - x0), 0.05*(x1 - x0))
            ), trans)
    trans = np.dot(np.array([[1, 0, 0.5*(y1 + y0)], [0, 1, 0.5*(x1 + x0)], [0, 0, 1]]), trans)

    # Read the image, transform to black and white and comvert to numpy array
    img   = read_raw_image(p).convert('L')
    img   = img_to_array(img)
    
    # Resize: Apply affine transformation
    matrix = trans[:2,:2]
    offset = trans[:2,2]
    img    = img.reshape(img.shape[:-1])
    img    = affine_transform(img, matrix, offset, output_shape=img_shape[:-1], order=1, mode='constant', cval=np.average(img))
    img    = img.reshape(img_shape)

    # Normalize to zero mean and unit variance
    img  -= np.mean(img, keepdims=True)
    img  /= np.std(img, keepdims=True) + K.epsilon()
    return img

def read_raw_image(p):
    # name to image 
    img = pil_image.open(expand_path(p))
    return img

def read_for_training(p):
    return read_cropped_image(p, False)

In [ ]:
def whether_new_whale(data,index):
    if data['Id'][index] == "new_whale":
        return True
    else:
        return False


def count_whale_id(data):
    count = 0
    for i in np.arange(0,data.shape[0]):
        if whether_new_whale(data,i) == False:
            count += 1
    return count



In [ ]:
train_test_df = pd.concat([train_new_df,test_new_df],ignore_index=True)
count_whale_train_new = count_whale_id(train_test_df)
print(count_whale_train_new) 
print(count_whale_id(train_new_df))
print(count_whale_id(test_new_df))


#count_whale_train_new = count_whale_id(train_test_df)
#print(count_whale_train_new) 
#print(count_whale_id(test_new_df))

969
646
323


In [ ]:
def num_class(data):
    unique_id = pd.unique(data['Id'])
    if "new_whale"  in unique_id:
        return len(unique_id)-1
    else:
        return(len(unique_id))

count_whale_id_all = num_class(train_test_df) 
print(count_whale_id_all) 
print(num_class(test_new_df))
print(num_class(train_new_df))

199
162
195


**CNN Model**

In [ ]:

def top_5_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=5)


def build_simple_cnn(l2,lr,img_shape=img_shape):
    regul = regularizers.l2(l2)
    optim = Adam(lr = lr)
    input = Input(shape=img_shape) 
    x = Concatenate()([input,input,input])
    base_model = ResNet50(include_top=False,input_tensor=x, weights='imagenet',pooling="max") #output layer 2048 
    out = Dense(count_whale_id_all,activation="softmax",name="softmax")(base_model.output)
    model = Model(input,out)
    model.compile(optim, loss = 'categorical_crossentropy', metrics=[categorical_crossentropy, 
                                                                     categorical_accuracy,top_5_accuracy])

    return model

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

def prepare_labels(y):    
    values = np.array(y)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    # print(integer_encoded)

    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    # print(onehot_encoded)

    y = onehot_encoded
    # print(y.shape)
    return y, label_encoder

In [ ]:
#select whale_id without new_whale

#train:290
all_whale_id_train_new = [train_new_df['Id'][i] for i in np.arange(0,train_new_df.shape[0]) if whether_new_whale(train_new_df,i)==False]
x_train_new =  [train_new_df['Image'][i] for i in np.arange(0,train_new_df.shape[0]) if whether_new_whale(train_new_df,i)==False]

#test:146
all_whale_id_test_new = [test_new_df['Id'][i] for i in np.arange(0,test_new_df.shape[0]) if whether_new_whale(test_new_df,i)==False]
x_test_new =  [test_new_df['Image'][i] for i in np.arange(0,test_new_df.shape[0]) if whether_new_whale(test_new_df,i)==False]


all_whale_id_train_test = all_whale_id_train_new + all_whale_id_test_new
all_y, all_label_encoder = prepare_labels(all_whale_id_train_test)
y_train_new = all_y[0:len(x_train_new),]
y_test_new = all_y[len(x_train_new):,]

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [ ]:
print(y_train_new.shape)
print(y_test_new.shape)

(646, 199)
(323, 199)


**Train CNN**

In [ ]:

# Here, `x_set` is list of path to the images: x
# and `y_set` are the associated classes: y

img_shape = (384,384,1)

class TrainingData(Sequence):
    def __init__(self,x_set,y_set,batch_size=32):
        super(TrainingData, self).__init__()
        #def __init__(self, x_set, y_set, batch_size=32):
        self.x, self.y = x_set, y_set  # x:list;y:np.array    
        self.batch_size = batch_size
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        start = self.batch_size*idx
        end   = min(start + self.batch_size, len(self.x))
        indexes = self.index[start:end]
        X_train,y_label = self.__data_generation(indexes)

        return X_train, y_label 

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.index = np.arange(len(self.x))
        #if self.shuffle == True:
        np.random.shuffle(self.index)
        
    
    def __data_generation(self, indexes):
        #print("data_generation")
        size  = indexes.shape[0]
        X_train = np.empty((size,)+img_shape, dtype=K.floatx())
        y_label = np.empty((size,self.y.shape[1]))        
       
        #y_label = self.y[indexes]
        for i  in  range(0,size): 
            #print(indexes[i])
            X_train[i,:,:,:] = read_for_training(self.x[indexes[i]])
            y_label[i,:] = self.y[indexes[i]]
        return X_train, y_label
    
     

In [ ]:
model = build_simple_cnn(0,10e-5) #l2 and lr
#try:l2 0 lr 10e-5 good
#.      10e-5 10e-5 bad
#              10e-8 learn too slow
#          0    10e-4 overfitting

Instructions for updating:
Colocations handled automatically by placer.
94658560/94653016 [==============================] - 3s 0us/step


**Start Training**

In [ ]:
train_new_data=TrainingData(x_train_new,y_train_new)
test_new_data=TrainingData(x_test_new,y_test_new)
model.fit_generator(train_new_data,validation_data=test_new_data,epochs=100)


Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/100
21/21 [==============================] - 35s 2s/step - loss: 15.1436 - categorical_crossentropy: 15.1436 - categorical_accuracy: 0.0239 - top_5_accuracy: 0.0522 - val_loss: 15.1176 - val_categorical_crossentropy: 15.1176 - val_categorical_accuracy: 0.0217 - val_top_5_accuracy: 0.0433
Epoch 2/100
21/21 [==============================] - 16s 744ms/step - loss: 13.5455 - categorical_crossentropy: 13.5455 - categorical_accuracy: 0.1252 - top_5_accuracy: 0.1536 - val_loss: 14.2668 - val_categorical_crossentropy: 14.2668 - val_categorical_accuracy: 0.0495 - val_top_5_accuracy: 0.1115
Epoch 3/100
21/21 [==============================] - 17s 811ms/step - loss: 12.3845 - categorical_crossentropy: 12.3845 - categorical_accuracy: 0.2134 - top_5_accuracy: 0.2313 - val_loss: 14.2438 - val_categorical_crossentropy: 14.2438 - val_categorical_accuracy: 0.0526 - val_

**Prediction**

In [ ]:
model.evaluate_generator(test_new_data,steps=1)

[10.19494342803955, 10.19494342803955, 0.28125, 0.40625]